In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats

from IPython.display import display
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'inline')

In [2]:
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier

from tqdm import tqdm 
from time import time

In [3]:
%run ../src/load_data.py

#### One Hot Encoding

In [4]:
housing_cat_df = housing_df.select_dtypes(include = ['category'])

In [5]:
dum_cat_df = pd.get_dummies(housing_cat_df,prefix_sep='_')

In [6]:
final_feature_df = pd.merge(dum_cat_df, numeric_log_sc_df, left_index=True, right_index=True)

In [7]:
final_feature_df.head()

,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,...,GarageYrBlt,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,ThreeSsnPorch,ScreenPorch,PoolArea,MiscVal,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,0,0,0,0,...,1.027172,0.352403,-0.752672,0.220671,-0.359239,-0.116664,-0.271013,-0.068882,-0.087931,0.138937
2,1,0,0,0,0,0,0,0,0,0,...,-0.098465,-0.058482,1.623025,-0.702730,-0.359239,-0.116664,-0.271013,-0.068882,-0.087931,-0.613295
3,0,0,0,0,0,1,0,0,0,0,...,0.943792,0.632552,-0.752672,-0.066946,-0.359239,-0.116664,-0.271013,-0.068882,-0.087931,0.138937
4,0,0,0,0,0,0,1,0,0,0,...,0.818721,0.791303,-0.752672,-0.172910,4.091881,-0.116664,-0.271013,-0.068882,-0.087931,-1.365527
5,0,0,0,0,0,1,0,0,0,0,...,0.902101,1.697119,0.777979,0.568838,-0.359239,-0.116664,-0.271013,-0.068882,-0.087931,0.138937


In [8]:
final_feature_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1451 entries, 1 to 1460
Columns: 382 entries, MSSubClass_20 to YrSold
dtypes: float64(23), uint8(359)
memory usage: 780.8 KB


#### Decision Tree

In [9]:
from sklearn.tree import DecisionTreeClassifier

In [10]:
dtree = DecisionTreeClassifier()

In [11]:
y = housing_df['SalePrice']

In [12]:
X = final_feature_df

In [13]:
dtree.fit(X,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [14]:
important_feature = dtree.feature_importances_
features = final_feature_df.columns
important_feature = pd.Series(important_feature.T.ravel(), index=features)

In [15]:
sort_imp_feat = important_feature.sort_values(ascending = False)
print ('\n Top 10 Features')
sort_imp_feat.head(10)


 Top 10 Features


GarageArea      0.032461
LotArea         0.029766
BsmtUnfSF       0.026746
FirstFlrSF      0.024795
TotalBsmtSF     0.024428
GarageYrBlt     0.022388
YrSold          0.021649
YearRemodAdd    0.020878
YearBuilt       0.019128
GrLivArea       0.018704
dtype: float64

#### Random Forest

In [16]:
from sklearn.ensemble import RandomForestClassifier

In [17]:
rfc = RandomForestClassifier(n_estimators=500)

In [18]:
rfc.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [19]:
rfc_important_feature = rfc.feature_importances_
rfc_features = final_feature_df.columns
rfc_important_feature = pd.Series(rfc_important_feature.T.ravel(), index=rfc_features)

In [20]:
sort_rfc_feat = rfc_important_feature.sort_values(ascending = False)
print ('\n Top 10 Features')
sort_rfc_feat.head(10)


 Top 10 Features


GrLivArea       0.024301
LotArea         0.024260
FirstFlrSF      0.023522
TotalBsmtSF     0.022983
GarageArea      0.022976
BsmtUnfSF       0.022948
LotFrontage     0.021239
YearBuilt       0.020814
GarageYrBlt     0.020649
YearRemodAdd    0.019792
dtype: float64

sortrfc_imp_feat = rfc_important_feature.sort_values(ascending = False)
print ('\n Top 10 Features')
sortrfc_imp_feat.head(20)

**Selecting Top 10 Features from RandomForestClassifier**

Create a new data frame of Topfeatures

In [34]:
feature_selected = ['GrLivArea','LotArea','FirstFlrSF','TotalBsmtSF','GarageArea','BsmtUnfSF','LotFrontage','YearBuilt','GarageYrBlt','YearRemodAdd']

In [35]:
housing_df[feature_selected]

,GrLivArea,LotArea,FirstFlrSF,TotalBsmtSF,GarageArea,BsmtUnfSF,LotFrontage,YearBuilt,GarageYrBlt,YearRemodAdd
Id,,,,,,,,,,
1,1710,8450,856,856,548,150,65.000000,2003,2003.000000,2003
2,1262,9600,1262,1262,460,284,80.000000,1976,1976.000000,1976
3,1786,11250,920,920,608,434,68.000000,2001,2001.000000,2002
4,1717,9550,961,756,642,540,60.000000,1915,1998.000000,1970
5,2198,14260,1145,1145,836,490,84.000000,2000,2000.000000,2000
6,1362,14115,796,796,480,64,85.000000,1993,1993.000000,1995
7,1694,10084,1694,1686,636,317,75.000000,2004,2004.000000,2005
8,2090,10382,1107,1107,484,216,70.049958,1973,1973.000000,1973
9,1774,6120,1022,952,468,952,51.000000,1931,1931.000000,1950


seems like lot of data are redundant, so reselect few more column

In [36]:
sort_rfc_feat = rfc_important_feature.sort_values(ascending = False)
print ('\n Top 10 Features')
sort_rfc_feat.head(20)


 Top 10 Features


GrLivArea           0.024301
LotArea             0.024260
FirstFlrSF          0.023522
TotalBsmtSF         0.022983
GarageArea          0.022976
BsmtUnfSF           0.022948
LotFrontage         0.021239
YearBuilt           0.020814
GarageYrBlt         0.020649
YearRemodAdd        0.019792
BsmtFinSF1          0.018862
OpenPorchSF         0.016803
WoodDeckSF          0.016111
YrSold              0.015535
MasVnrArea          0.014225
SecondFlrSF         0.013146
LotShape_Reg        0.006246
LotShape_IR1        0.006227
TotRmsAbvGrd_6      0.006087
LotConfig_Inside    0.006027
dtype: float64

In [37]:
final_feature_selected = ['GrLivArea','LotArea','FirstFlrSF','GarageArea','BsmtUnfSF','LotFrontage','YearBuilt','YearRemodAdd','BsmtFinSF1','OpenPorchSF']

In [38]:
housing_df[final_feature_selected]

,GrLivArea,LotArea,FirstFlrSF,GarageArea,BsmtUnfSF,LotFrontage,YearBuilt,YearRemodAdd,BsmtFinSF1,OpenPorchSF
Id,,,,,,,,,,
1,1710,8450,856,548,150,65.000000,2003,2003,706,61
2,1262,9600,1262,460,284,80.000000,1976,1976,978,0
3,1786,11250,920,608,434,68.000000,2001,2002,486,42
4,1717,9550,961,642,540,60.000000,1915,1970,216,35
5,2198,14260,1145,836,490,84.000000,2000,2000,655,84
6,1362,14115,796,480,64,85.000000,1993,1995,732,30
7,1694,10084,1694,636,317,75.000000,2004,2005,1369,57
8,2090,10382,1107,484,216,70.049958,1973,1973,859,204
9,1774,6120,1022,468,952,51.000000,1931,1950,0,0


Will do PCA using above data in next section